In [19]:
import pandas as pd
import requests
import os
import re
import time
import json
from urllib.parse import urlparse, urljoin
from pathlib import Path
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.options import Options
from selenium.common.exceptions import TimeoutException, NoSuchElementException
from bs4 import BeautifulSoup

class SamsungWasherDryerScraper:
    def __init__(self):
        self.base_url = "https://www.samsung.com/sec/washers-and-dryers/all-washers-and-dryers/"
        self.images_folder = './samsung_img'
        self.manuals_folder = './samsung_manuals'
        self.csv_file = 'samsung_washer_dryer_products.csv'
        self.setup_folders()
        self.driver = None

    def setup_folders(self):
        """이미지와 매뉴얼 다운로드 폴더 생성"""
        for folder in [self.images_folder, self.manuals_folder]:
            if not os.path.exists(folder):
                os.makedirs(folder)
                print(f"📁 폴더 생성: {folder}")
            else:
                print(f"📁 폴더 확인: {folder}")

    def setup_driver(self):
        """Selenium 웹드라이버 설정"""
        print("🚀 웹드라이버 설정 시작...")
        chrome_options = Options()
        # chrome_options.add_argument("--headless")  # 디버깅을 위해 비활성화
        chrome_options.add_argument("--no-sandbox")
        chrome_options.add_argument("--disable-dev-shm-usage")
        chrome_options.add_argument("--disable-gpu")
        chrome_options.add_argument("--window-size=1920,1080")
        chrome_options.add_argument("--user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36")

        # 다운로드 설정
        prefs = {
            "download.default_directory": os.path.abspath(self.manuals_folder),
            "download.prompt_for_download": False,
            "download.directory_upgrade": True,
            "safebrowsing.enabled": True,
            "profile.default_content_settings.popups": 0,
            "profile.default_content_setting_values.automatic_downloads": 1
        }
        chrome_options.add_experimental_option("prefs", prefs)

        self.driver = webdriver.Chrome(options=chrome_options)
        self.driver.implicitly_wait(10)
        print("✅ 웹드라이버 초기화 완료")

    def close_driver(self):
        """웹드라이버 종료"""
        if self.driver:
            self.driver.quit()
            print("🛑 웹드라이버 종료")

    def handle_out_of_stock_filter(self):
        """품절상품 제외 필터 해제"""
        print("🔍 품절상품 제외 버튼 탐색 중...")

        try:
            exclude_sold_out_label = WebDriverWait(self.driver, 15).until(
                EC.presence_of_element_located((By.CSS_SELECTOR, "label[for='toggle-check-on-pc']"))
            )
            print("   ✅ 품절상품 제외 체크박스 라벨 감지")

            exclude_sold_out_checkbox = self.driver.find_element(By.CSS_SELECTOR, "input#toggle-check-on-pc")

            if exclude_sold_out_checkbox.is_selected():
                exclude_sold_out_label.click()
                print("   ✅ 품절상품 제외 체크박스 해제 완료")
            else:
                print("   ℹ️  품절상품 제외 체크박스가 이미 해제되어 있음")

            time.sleep(2)

        except Exception as e:
            print(f"   ❌ 품절상품 제외 체크박스 처리 실패: {str(e)}")

    def scroll_and_load_all_products(self):
        """더보기 버튼 클릭으로 모든 제품 로드"""
        print("📜 더보기 버튼 클릭으로 모든 제품 로드 시작...")

        click_count = 0
        max_clicks = 10

        while click_count < max_clicks:
            try:
                more_button = WebDriverWait(self.driver, 5).until(
                    EC.element_to_be_clickable((By.CSS_SELECTOR, "button.btn.btn-type1.btn-d.btn-readmore"))
                )

                print(f"   [{click_count + 1}] ✅ 더보기 버튼 감지")

                try:
                    current_page_elem = self.driver.find_element(By.ID, "presentPageCount")
                    total_pages_elem = self.driver.find_element(By.ID, "totalPageCount")

                    current_page_text = current_page_elem.text.strip()
                    total_pages_text = total_pages_elem.text.strip()

                    if current_page_text and total_pages_text:
                        current_page = int(current_page_text)
                        total_pages = int(total_pages_text)
                        print(f"      📄 현재 페이지: {current_page}/{total_pages}")

                        if current_page >= total_pages:
                            print("      ✅ 마지막 페이지에 도달했습니다.")
                            break

                except Exception as e:
                    print(f"      ⚠️  페이지 번호 확인 실패: {str(e)} - 계속 진행")

                if not more_button.is_displayed() or not more_button.is_enabled():
                    print("      ❌ 더보기 버튼이 클릭할 수 없는 상태")
                    break

                self.driver.execute_script("arguments[0].click();", more_button)
                print(f"      🖱️  더보기 버튼 클릭 완료")
                click_count += 1

                time.sleep(3)

                try:
                    new_more_button = self.driver.find_element(By.CSS_SELECTOR, "button.btn.btn-type1.btn-d.btn-readmore")
                    self.driver.execute_script("arguments[0].scrollIntoView();", new_more_button)
                except:
                    self.driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")

                time.sleep(2)

            except TimeoutException:
                print(f"   ✅ 더보기 버튼을 찾을 수 없음 - 모든 제품 로드 완료")
                break
            except Exception as e:
                print(f"   ❌ 더보기 버튼 처리 실패: {str(e)}")
                break

        print(f"✅ 더보기 버튼 클릭 완료. 총 {click_count}번 클릭")
        self.driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(3)
        print("🎯 모든 제품 로딩 완료!")

    def extract_product_names_from_html(self, parent_element):
        """HTML에서 한글 모델명과 영문 모델명 추출"""
        print(f"      🔍 HTML에서 모델명 추출 시도...")

        korean_name = ""
        english_code = ""

        try:
            # 한글 모델명 찾기: prd-name 클래스
            korean_element = parent_element.select_one('span.prd-name')
            if korean_element:
                korean_name = korean_element.get_text(strip=True)
                print(f"         📋 한글 모델명: {korean_name}")

            # 영문 모델명 찾기: prd-num 클래스
            english_element = parent_element.select_one('span.prd-num')
            if english_element:
                english_code = english_element.get_text(strip=True)
                print(f"         📋 영문 모델명: {english_code}")

            # 대체 방법: 다른 선택자들도 시도
            if not korean_name:
                # title 속성에서 찾기
                title_elements = parent_element.select('[title]')
                for elem in title_elements:
                    title = elem.get('title', '').strip()
                    if title and any(keyword in title for keyword in ['Bespoke', 'AI', '콤보', 'kg']):
                        korean_name = title
                        print(f"         📋 한글 모델명 (title): {korean_name}")
                        break

            if not english_code:
                # 영문 코드 패턴으로 찾기
                all_text = parent_element.get_text()
                model_patterns = [
                    r'WD[0-9A-Z]{8,}',
                    r'WF[0-9A-Z]{8,}',
                    r'DV[0-9A-Z]{8,}',
                    r'WA[0-9A-Z]{8,}'
                ]
                for pattern in model_patterns:
                    matches = re.findall(pattern, all_text)
                    if matches:
                        english_code = matches[0]
                        print(f"         📋 영문 모델명 (패턴): {english_code}")
                        break

        except Exception as e:
            print(f"         ⚠️ 모델명 추출 오류: {str(e)}")

        return korean_name, english_code

    def generate_filename_from_names(self, korean_name, english_code, extension):
        """한글모델명_영문모델명.확장자 형태로 파일명 생성"""
        print(f"      📝 파일명 생성 중...")
        print(f"         한글명: {korean_name}")
        print(f"         영문코드: {english_code}")

        # 한글명 정리 (공백을 _로 변경, 특수문자 제거)
        if korean_name:
            clean_korean = re.sub(r'[^\w\s가-힣]', '', korean_name)
            clean_korean = re.sub(r'\s+', '_', clean_korean.strip())
        else:
            clean_korean = "Unknown_Product"

        # 영문코드 정리 (공백을 _로 변경)
        if english_code:
            clean_english = re.sub(r'\s+', '_', english_code.strip())
        else:
            clean_english = "UNKNOWN"

        # 최종 파일명: 한글명_영문코드.확장자
        filename = f"{clean_korean}_{clean_english}{extension}"

        # 파일시스템에서 사용할 수 없는 문자 추가 제거
        filename = re.sub(r'[<>:"/\\|?*]', '_', filename)

        # 파일명이 너무 길면 자르기 (Windows 파일명 제한)
        if len(filename) > 200:
            korean_part = clean_korean[:80]
            english_part = clean_english[:50]
            filename = f"{korean_part}_{english_part}{extension}"

        print(f"         생성된 파일명: {filename}")
        return filename

    def get_top20_products_improved(self):
        """개선된 상위 20개 제품 정보 수집"""
        print("📊 상위 20개 제품 정보 수집 중...")

        try:
            html = self.driver.page_source
            soup = BeautifulSoup(html, 'html.parser')

            # 더 많은 선택자 시도
            product_selectors = [
                'div.item-inner a[href*="washers-and-dryers"]',
                'div.item-inner > div.card-detail > a',
                '.product-card a',
                '.pd-item a',
                '.product-item a',
                'a[href*="/washers-and-dryers/"]',
                'div.card-detail a'
            ]

            product_elements = []
            for selector in product_selectors:
                elements = soup.select(selector)
                if elements:
                    product_elements = elements
                    print(f"   ✅ 제품 요소 발견: {selector} ({len(elements)}개)")
                    break

            if not product_elements:
                print("   ❌ 제품 요소를 찾을 수 없음")
                return []

            products = []
            max_products = min(20, len(product_elements))
            print(f"   📦 상위 {max_products}개 제품 처리 시작...")

            for i in range(max_products):
                try:
                    element = product_elements[i]
                    print(f"\n   [{i+1:2d}] 제품 정보 추출 중...")

                    # href 추출 및 정리
                    href = element.get('href', '')
                    if "'" in href:
                        relative_link = href.split("'")[1]
                    else:
                        relative_link = href

                    if not relative_link.startswith('http'):
                        product_link = urljoin("https://www.samsung.com", relative_link)
                    else:
                        product_link = relative_link

                    print(f"        완전한 URL: {product_link}")

                    # 부모 요소에서 한글/영문 모델명 추출
                    parent_div = element.find_parent('div', class_='item-inner')
                    if parent_div:
                        korean_name, english_code = self.extract_product_names_from_html(parent_div)
                    else:
                        korean_name, english_code = "", ""

                    # 이미지 URL 추출 개선
                    image_url = ""
                    if parent_div:
                        img_elements = parent_div.find_all('img')
                        for img in img_elements:
                            src = img.get('src') or img.get('data-src') or img.get('data-original') or ""

                            # Samsung 제품 이미지 확인
                            if (src and
                                ('images.samsung.com' in src or '/kdp/goods' in src) and
                                'useinsider.com' not in src and
                                'banner' not in src.lower() and
                                'badge' not in src.lower()):

                                if src.startswith('//'):
                                    image_url = 'https:' + src
                                elif src.startswith('/'):
                                    image_url = urljoin("https://www.samsung.com", src)
                                else:
                                    image_url = src

                                # 고해상도 이미지로 변경
                                if '?$' in image_url:
                                    base_url = image_url.split('?$')[0]
                                    image_url = f"{base_url}?$PF_PRD_KDP_PNG$"

                                print(f"        이미지 URL: {image_url}")
                                break

                    if product_link and (korean_name or english_code):
                        product_info = {
                            'index': i+1,
                            'korean_name': korean_name,
                            'english_code': english_code,
                            'product_link': product_link,
                            'image_url': image_url
                        }
                        products.append(product_info)
                        print(f"        ✅ 제품 정보 추가 완료")
                    else:
                        print(f"        ❌ 필수 정보 누락")

                except Exception as e:
                    print(f"        ❌ 제품 {i+1} 정보 추출 실패: {str(e)}")
                    continue

            print(f"\n✅ {len(products)}개 제품 정보 추출 완료")
            return products

        except Exception as e:
            print(f"❌ 제품 정보 추출 중 오류: {str(e)}")
            return []

    def download_image_improved(self, image_url, filename):
        """개선된 이미지 다운로드"""
        if not image_url:
            print(f"      ❌ 이미지 URL이 없음")
            return False

        try:
            print(f"      📥 이미지 다운로드 시작...")

            # URL 정리
            if image_url.startswith('//'):
                image_url = 'https:' + image_url
            elif image_url.startswith('/'):
                image_url = 'https://images.samsung.com' + image_url

            print(f"         최종 URL: {image_url}")

            headers = {
                'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36',
                'Accept': 'image/webp,image/apng,image/*,*/*;q=0.8',
                'Accept-Language': 'ko-KR,ko;q=0.9,en;q=0.8',
                'Referer': 'https://www.samsung.com/',
                'Connection': 'keep-alive'
            }

            response = requests.get(image_url, headers=headers, timeout=30, stream=True)
            print(f"         HTTP 응답 코드: {response.status_code}")

            if response.status_code == 200:
                file_path = os.path.join(self.images_folder, filename)

                with open(file_path, 'wb') as f:
                    for chunk in response.iter_content(chunk_size=8192):
                        if chunk:
                            f.write(chunk)

                file_size = os.path.getsize(file_path)
                if file_size > 1000:  # 1KB 이상인 경우만 성공으로 간주
                    print(f"         ✅ 이미지 다운로드 완료: {filename}")
                    print(f"         📊 파일 크기: {file_size:,} bytes")
                    return True
                else:
                    print(f"         ❌ 파일 크기가 너무 작음: {file_size} bytes")
                    os.remove(file_path)
                    return False
            else:
                print(f"         ❌ HTTP 오류: {response.status_code}")
                return False

        except Exception as e:
            print(f"         ❌ 이미지 다운로드 오류: {str(e)}")
            return False

    def download_all_images_improved(self, products):
        """개선된 이미지 다운로드"""
        print("\n📸 상위 20개 제품 이미지 다운로드 시작...")
        print("=" * 50)

        success_count = 0

        for product in products:
            try:
                print(f"\n🖼️  [{product['index']:2d}/{len(products)}] 이미지 다운로드")
                print(f"      한글명: {product['korean_name'][:30]}...")
                print(f"      영문코드: {product['english_code']}")

                # 확장자 결정
                image_url = product['image_url']
                if not image_url:
                    print(f"      ❌ 이미지 URL이 없음")
                    continue

                if '.png' in image_url.lower():
                    extension = '.png'
                elif '.webp' in image_url.lower():
                    extension = '.webp'
                else:
                    extension = '.jpg'

                # 새로운 파일명 생성: 한글명_영문코드.확장자
                filename = self.generate_filename_from_names(
                    product['korean_name'],
                    product['english_code'],
                    extension
                )

                # 이미 파일이 존재하는지 확인
                file_path = os.path.join(self.images_folder, filename)
                if os.path.exists(file_path):
                    print(f"      ⚠️ 이미지 파일이 이미 존재: {filename}")
                    success_count += 1
                    continue

                if self.download_image_improved(product['image_url'], filename):
                    success_count += 1

                time.sleep(1)

            except Exception as e:
                print(f"      ❌ 이미지 다운로드 실패: {str(e)}")

        print(f"\n✅ 이미지 다운로드 완료: {success_count}/{len(products)}개")
        return success_count

    def find_manual_section_and_extract_names(self):
        """매뉴얼 섹션에서 한글/영문 모델명 추출"""
        print(f"      🔍 매뉴얼 섹션에서 모델명 추출...")

        korean_name = ""
        english_code = ""

        try:
            # 매뉴얼 섹션 버튼 찾기 및 클릭
            manual_button_selectors = [
                "//span[contains(text(), '매뉴얼')]",
                "//button[contains(., '매뉴얼')]",
                "//a[contains(., '매뉴얼')]",
                "//*[contains(text(), 'Manual')]"
            ]

            manual_clicked = False
            for selector in manual_button_selectors:
                try:
                    elements = self.driver.find_elements(By.XPATH, selector)
                    for element in elements:
                        if element.is_displayed():
                            print(f"         ✅ 매뉴얼 섹션 버튼 발견: {element.text}")
                            self.driver.execute_script("arguments[0].scrollIntoView(true);", element)
                            time.sleep(1)
                            element.click()
                            time.sleep(3)
                            manual_clicked = True
                            break
                    if manual_clicked:
                        break
                except Exception:
                    continue

            if manual_clicked:
                print(f"         ✅ 매뉴얼 섹션 열기 성공")

                # 제품 정보 추출: <p class="info-prd"><strong>한글명</strong>영문코드</p>
                try:
                    info_prd_elements = self.driver.find_elements(By.CSS_SELECTOR, "p.info-prd")

                    for info_elem in info_prd_elements:
                        try:
                            # strong 태그 안의 한글명
                            strong_elem = info_elem.find_element(By.TAG_NAME, "strong")
                            korean_name = strong_elem.text.strip()

                            # 전체 텍스트에서 strong 텍스트를 제외한 나머지가 영문코드
                            full_text = info_elem.text.strip()
                            english_code = full_text.replace(korean_name, "").strip()

                            if korean_name and english_code:
                                print(f"         📋 매뉴얼에서 한글명: {korean_name}")
                                print(f"         📋 매뉴얼에서 영문코드: {english_code}")
                                break

                        except Exception:
                            continue

                except Exception as e:
                    print(f"         ⚠️ info-prd에서 모델명 추출 실패: {str(e)}")

        except Exception as e:
            print(f"         ⚠️ 매뉴얼 섹션 처리 오류: {str(e)}")

        return korean_name, english_code

    def find_user_manual_download_link(self):
        """사용자 매뉴얼 다운로드 링크 찾기"""
        print(f"         🔍 사용자 매뉴얼 탐색...")

        try:
            # 사용자 매뉴얼을 포함하는 strong 요소 찾기
            user_manual_elements = self.driver.find_elements(By.XPATH, "//strong[@class='name'][contains(text(), '사용자 매뉴얼')]")

            if not user_manual_elements:
                # 대체 선택자들
                user_manual_elements = self.driver.find_elements(By.XPATH, "//strong[contains(text(), '사용자 매뉴얼')]")

            for strong_elem in user_manual_elements:
                try:
                    print(f"            ✅ 사용자 매뉴얼 발견: {strong_elem.text}")

                    # 부모 li 요소 찾기
                    li_element = strong_elem.find_element(By.XPATH, "./ancestor::li")

                    # li 안의 다운로드 버튼 찾기
                    download_links = li_element.find_elements(By.CSS_SELECTOR, "a.btn-download")

                    for download_link in download_links:
                        href = download_link.get_attribute('href')
                        data_file = download_link.get_attribute('data-nmfile') or ""

                        if href and '.pdf' in href.lower():
                            print(f"            📄 사용자 매뉴얼 다운로드 링크: {data_file}")
                            return {
                                'url': href,
                                'filename': data_file
                            }

                except Exception as e:
                    print(f"            ⚠️ 사용자 매뉴얼 처리 오류: {str(e)}")
                    continue

            # 사용자 매뉴얼을 못 찾으면 첫 번째 매뉴얼 사용
            print(f"         ⚠️ 사용자 매뉴얼을 찾을 수 없음, 첫 번째 매뉴얼 사용")

            all_download_links = self.driver.find_elements(By.CSS_SELECTOR, "a.btn-download")
            for download_link in all_download_links:
                href = download_link.get_attribute('href')
                data_file = download_link.get_attribute('data-nmfile') or ""

                if href and '.pdf' in href.lower():
                    print(f"         📄 첫 번째 매뉴얼 다운로드 링크: {data_file}")
                    return {
                        'url': href,
                        'filename': data_file
                    }

            return None

        except Exception as e:
            print(f"         ❌ 매뉴얼 링크 찾기 오류: {str(e)}")
            return None

    def download_pdf_improved(self, korean_name, english_code, product_url):
        """개선된 PDF 다운로드"""
        print(f"   🔍 PDF 다운로드 프로세스 시작...")
        print(f"      한글명: {korean_name}")
        print(f"      영문코드: {english_code}")
        print(f"      URL: {product_url}")

        try:
            self.driver.get(product_url)
            time.sleep(5)

            # 페이지 끝까지 스크롤
            self.driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
            time.sleep(3)

            # 매뉴얼 섹션에서 모델명 추출 (더 정확한 정보 가능)
            manual_korean, manual_english = self.find_manual_section_and_extract_names()

            # 매뉴얼에서 추출한 모델명이 있으면 사용, 없으면 기존 정보 사용
            final_korean = manual_korean if manual_korean else korean_name
            final_english = manual_english if manual_english else english_code

            print(f"      최종 한글명: {final_korean}")
            print(f"      최종 영문코드: {final_english}")

            # 사용자 매뉴얼 다운로드 링크 찾기
            download_info = self.find_user_manual_download_link()

            if download_info:
                try:
                    print(f"      📥 PDF 다운로드 시작...")
                    print(f"         원본 파일명: {download_info['filename']}")

                    response = requests.get(download_info['url'],
                                          timeout=60,
                                          headers={'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36'})

                    if response.status_code == 200:
                        # 새로운 파일명 생성: 한글명_영문코드.pdf
                        filename = self.generate_filename_from_names(
                            final_korean,
                            final_english,
                            '.pdf'
                        )

                        pdf_path = os.path.join(self.manuals_folder, filename)

                        with open(pdf_path, 'wb') as f:
                            f.write(response.content)

                        file_size = len(response.content)
                        if file_size > 10000:  # 10KB 이상인 경우만 성공으로 간주
                            print(f"         ✅ 다운로드 성공: {filename}")
                            print(f"         📊 파일 크기: {file_size:,} bytes")
                            return True
                        else:
                            print(f"         ❌ 파일 크기가 너무 작음")
                            os.remove(pdf_path)
                            return False
                    else:
                        print(f"         ❌ HTTP 오류: {response.status_code}")
                        return False

                except Exception as e:
                    print(f"         ❌ 다운로드 실패: {str(e)}")
                    return False
            else:
                print(f"      ❌ 매뉴얼 링크를 찾을 수 없음")
                return False

        except Exception as e:
            print(f"      ❌ PDF 다운로드 프로세스 오류: {str(e)}")
            return False

    def download_all_manuals_improved(self, products):
        """개선된 매뉴얼 다운로드"""
        print("\n📚 상위 20개 제품 매뉴얼 다운로드 시작...")
        print("=" * 50)

        success_count = 0

        for product in products:
            try:
                print(f"\n📄 [{product['index']:2d}/{len(products)}] 매뉴얼 다운로드")
                print(f"      한글명: {product['korean_name'][:30]}...")
                print(f"      영문코드: {product['english_code']}")

                # 매뉴얼 파일이 이미 존재하는지 확인
                manual_filename = self.generate_filename_from_names(
                    product['korean_name'],
                    product['english_code'],
                    '.pdf'
                )
                manual_path = os.path.join(self.manuals_folder, manual_filename)

                if os.path.exists(manual_path):
                    print(f"      ⚠️ 매뉴얼 파일이 이미 존재: {manual_filename}")
                    success_count += 1
                    continue

                if self.download_pdf_improved(
                    product['korean_name'],
                    product['english_code'],
                    product['product_link']
                ):
                    success_count += 1

                time.sleep(3)  # 서버 부하 방지

            except Exception as e:
                print(f"      ❌ 매뉴얼 다운로드 실패: {str(e)}")

        print(f"\n✅ 매뉴얼 다운로드 완료: {success_count}/{len(products)}개")
        return success_count

    def save_products_to_csv(self, products):
        """제품 정보를 CSV 파일로 저장"""
        print(f"\n💾 제품 정보 CSV 저장 중...")

        csv_data = []
        for product in products:
            csv_data.append({
                'index': product['index'],
                'korean_name': product['korean_name'],
                'english_code': product['english_code'],
                'product_url': product['product_link'],
                'image_url': product['image_url']
            })

        df = pd.DataFrame(csv_data)
        df.to_csv(self.csv_file, index=False, encoding='utf-8-sig')
        print(f"✅ 제품 정보 CSV 저장 완료: {self.csv_file} ({len(products)}개)")

    def run_scraping(self):
        """전체 스크래핑 프로세스 실행"""
        try:
            print("🚀 삼성 세탁기/건조기 스크래핑 시작!")
            print("=" * 60)

            self.setup_driver()

            print(f"\n🌐 페이지 접속: {self.base_url}")
            self.driver.get(self.base_url)
            time.sleep(5)

            self.handle_out_of_stock_filter()
            self.scroll_and_load_all_products()

            # 개선된 제품 정보 수집
            products = self.get_top20_products_improved()

            if not products:
                print("❌ 추출된 제품이 없습니다.")
                return

            self.save_products_to_csv(products)

            # 개선된 이미지 다운로드
            image_success = self.download_all_images_improved(products)

            # 개선된 매뉴얼 다운로드
            manual_success = self.download_all_manuals_improved(products)

            # 결과 요약
            print("\n" + "=" * 60)
            print("🎉 모든 작업 완료!")
            print(f"📊 최종 결과:")
            print(f"   - 처리된 제품 수: {len(products)}개")
            print(f"   - 이미지 다운로드: {image_success}개")
            print(f"   - 매뉴얼 다운로드: {manual_success}개")
            print(f"   - 이미지 폴더: {self.images_folder}")
            print(f"   - 매뉴얼 폴더: {self.manuals_folder}")
            print(f"   - 제품 정보 CSV: {self.csv_file}")

            # 폴더 내 파일 목록 확인
            try:
                image_files = [f for f in os.listdir(self.images_folder)
                             if f.lower().endswith(('.png', '.jpg', '.jpeg', '.webp'))]
                manual_files = [f for f in os.listdir(self.manuals_folder)
                              if f.lower().endswith('.pdf')]

                print(f"\n📂 다운로드된 파일 현황:")
                print(f"   - 이미지 파일: {len(image_files)}개")
                print(f"   - 매뉴얼 파일: {len(manual_files)}개")

                if image_files:
                    print(f"\n🖼️ 이미지 파일 예시:")
                    for i, filename in enumerate(image_files[:5], 1):
                        file_path = os.path.join(self.images_folder, filename)
                        file_size = os.path.getsize(file_path)
                        print(f"   {i}. {filename} ({file_size:,} bytes)")

                if manual_files:
                    print(f"\n📚 매뉴얼 파일 예시:")
                    for i, filename in enumerate(manual_files[:5], 1):
                        file_path = os.path.join(self.manuals_folder, filename)
                        file_size = os.path.getsize(file_path)
                        print(f"   {i}. {filename} ({file_size:,} bytes)")

            except Exception as e:
                print(f"   ⚠️ 폴더 확인 실패: {str(e)}")

        except Exception as e:
            print(f"❌ 스크래핑 중 오류 발생: {str(e)}")
            import traceback
            traceback.print_exc()

        finally:
            self.close_driver()

def main():
    scraper = SamsungWasherDryerScraper()
    scraper.run_scraping()

if __name__ == "__main__":
    main()

📁 폴더 확인: ./samsung_img
📁 폴더 확인: ./samsung_manuals
🚀 삼성 세탁기/건조기 스크래핑 시작!
🚀 웹드라이버 설정 시작...
✅ 웹드라이버 초기화 완료

🌐 페이지 접속: https://www.samsung.com/sec/washers-and-dryers/all-washers-and-dryers/
🔍 품절상품 제외 버튼 탐색 중...
   ✅ 품절상품 제외 체크박스 라벨 감지
   ✅ 품절상품 제외 체크박스 해제 완료
📜 더보기 버튼 클릭으로 모든 제품 로드 시작...
   [1] ✅ 더보기 버튼 감지
      📄 현재 페이지: 1/4
      🖱️  더보기 버튼 클릭 완료
   [2] ✅ 더보기 버튼 감지
      📄 현재 페이지: 2/4
      🖱️  더보기 버튼 클릭 완료
   [3] ✅ 더보기 버튼 감지
      📄 현재 페이지: 3/4
      🖱️  더보기 버튼 클릭 완료
   ✅ 더보기 버튼을 찾을 수 없음 - 모든 제품 로드 완료
✅ 더보기 버튼 클릭 완료. 총 3번 클릭
🎯 모든 제품 로딩 완료!
📊 상위 20개 제품 정보 수집 중...
   ✅ 제품 요소 발견: div.item-inner > div.card-detail > a (102개)
   📦 상위 20개 제품 처리 시작...

   [ 1] 제품 정보 추출 중...
        완전한 URL: https://www.samsung.com/sec/laundry-combo/combo-wd80f25chy-d2c/WD80F25CHY/
      🔍 HTML에서 모델명 추출 시도...
         📋 한글 모델명: Bespoke AI 콤보 25/15kg (109.2mm LCD)
         📋 영문 모델명: WD80F25CHY
        이미지 URL: https://images.samsung.com/kdp/goods/2025/05/14/c39fa9e9-daa4-4709-854f-549b60ee632e.png?$PF_PRD_KDP_PNG$